In [0]:
import numpy as np
import pandas as pd

from google.colab import files
uploaded = files.upload()

Saving data14smaller_0.csv to data14smaller_0 (1).csv


In [0]:
data18smaller = pd.read_csv("data14smaller_0.csv", low_memory = False, dtype = {'subsidies_flag':"int64"})


In [0]:
!pip install pgmpy==0.1.7
from pgmpy.estimators import HillClimbSearch, BicScore, BayesianEstimator
from pgmpy.models import BayesianModel
from sklearn.model_selection import train_test_split
from pgmpy.inference import VariableElimination

     |████████████████████████████████| 225kB 2.9MB/s 
     |████████████████████████████████| 1.3MB 43.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/27/d5/45/f9bd7e0f7d3752b706163ce2a0333d29855828c3e57a003640
Successfully built pgmpy
  Found existing installation: networkx 2.2
    Uninstalling networkx-2.2:
      Successfully uninstalled networkx-2.2


In [0]:
data18train, data18test = train_test_split(data18smaller,random_state=9,test_size=0.2)
hc = HillClimbSearch(data18train, scoring_method=BicScore(data18train))
best_model = hc.estimate()
print(best_model.edges())

[('bldgclass', 'subsidies_flag'), ('bldgclass', 'ownertype'), ('bldgclass', 'residfar'), ('bldgclass', 'yearbuilt'), ('bldgclass', 'violations'), ('bldgclass', 'landuse'), ('residfar', 'violations'), ('residfar', 'subsidies_flag'), ('violations', 'yearbuilt'), ('violations', 'subsidies_flag'), ('violations', 'ownertype'), ('violations', 'unitsres'), ('landuse', 'residfar'), ('landuse', 'violations'), ('landuse', 'unitsres')]


In [0]:
best_model.fit(data18train, estimator=BayesianEstimator, prior_type='K2')

for cpd in best_model.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)
    
print best_model.local_independencies('violations')

for thenode in best_model.nodes():
    print best_model.active_trail_nodes(thenode)

CPD of unitsres:
+-------------+------------------------+------------------------+------------------------+-----------------------+-----------------------+------------------------+------------------------+-----------------------+
| landuse     | landuse(0)             | landuse(0)             | landuse(1)             | landuse(1)            | landuse(2)            | landuse(2)             | landuse(3)             | landuse(3)            |
+-------------+------------------------+------------------------+------------------------+-----------------------+-----------------------+------------------------+------------------------+-----------------------+
| violations  | violations(0)          | violations(1)          | violations(0)          | violations(1)         | violations(0)         | violations(1)          | violations(0)          | violations(1)         |
+-------------+------------------------+------------------------+------------------------+-----------------------+-----------------

In [0]:
from collections import defaultdict
from pgmpy.inference import VariableElimination
def temp_predict(model,data):
    missing_variables = list(set(model.nodes()) - set(data.columns))
    pred_values = defaultdict(list)
    model_inference = VariableElimination(model)
    count = 0
    for index, data_point in data.iterrows():
        count = count+1
        if count %1000 == 1:
          print count
        states_dict = model_inference.map_query(variables=missing_variables, evidence=data_point.astype(int).to_dict())
        for k, v in states_dict.items():
            pred_values[k].append(v)
    return pd.DataFrame(pred_values, index=data.index)

In [0]:
best_model.fit(data18train, estimator=BayesianEstimator, prior_type='K2')
test_x = data18test.drop(columns=['violations'])
predicted_test = temp_predict(best_model, test_x)
print "Out of sample:",(data18test.loc[:,'violations']==predicted_test['violations']).mean()

Out of sample: 0.8827111323526808


In [0]:

train_x = data18train.drop(columns=['violations'])
predicted_train = temp_predict(best_model, train_x)
print "In of sample:",(data18train.loc[:,'violations']==predicted_train['violations']).mean()

In [0]:
data18smaller = pd.read_csv("data14smaller_2.csv", low_memory = False, dtype = {'subsidies_flag':"int64"})


In [0]:
# new indicator
data18train, data18test = train_test_split(data18smaller,random_state=9,test_size=0.2)
hc = HillClimbSearch(data18train, scoring_method=BicScore(data18train))
best_model = hc.estimate()
print(best_model.edges())

[('bldgclass', 'violations'), ('bldgclass', 'landuse'), ('bldgclass', 'ownertype'), ('bldgclass', 'subsidies_flag'), ('residfar', 'unit_ratio'), ('residfar', 'violations'), ('residfar', 'landuse'), ('residfar', 'yearbuilt'), ('residfar', 'bldgclass'), ('yearbuilt', 'violations'), ('yearbuilt', 'ownertype'), ('yearbuilt', 'subsidies_flag'), ('yearbuilt', 'bldgclass'), ('unit_ratio', 'yearbuilt'), ('unit_ratio', 'bldgclass'), ('violations', 'subsidies_flag'), ('violations', 'landuse')]


In [0]:
best_model.fit(data18train, estimator=BayesianEstimator, prior_type='K2')

for cpd in best_model.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)
    
print best_model.local_independencies('violations')

for thenode in best_model.nodes():
    print best_model.active_trail_nodes(thenode)

CPD of bldgclass:
+--------------+-----------------------+----------------------+------------------------+----------------------+----------------------+-----------------------+-----------------------+-----------------------+----------------------+-----------------------+----------------------+-----------------------+---------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+----------------------+-----------------------+----------------------+-----------------------+-----------------------+----------------------+----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+---------------+-----------------------+-----------------------+-----------------------+-----------------------+----------------------+-----------------------+-------

In [0]:
best_model.fit(data18train, estimator=BayesianEstimator, prior_type='K2')
test_x = data18test.drop(columns=['violations'])
predicted_test = temp_predict(best_model, test_x)
print "Out of sample:",(data18test.loc[:,'violations']==predicted_test['violations']).mean()

1
1001
2001
3001
4001
5001
6001
7001
8001
9001
10001
11001
12001
13001
14001
15001
16001
17001
18001
19001
20001
21001
22001
23001
24001
25001
26001
27001
28001
29001
30001
31001
32001
33001
34001
35001
36001
37001
38001
39001
40001
41001
42001
43001
44001
45001
46001
47001
48001
49001
50001
51001
52001
53001
54001
55001
56001
Out of sample: 0.8961818278370771
